In [3]:
#import requied packages
from gurobipy import *
import random
import pandas as pd
import numpy as np
#initialize data frames
results = pd.DataFrame(columns=['Instance', 'Tightness', 'Alpha','Makespan'])
instances = pd.DataFrame(columns=['units', 'nodes', 'arcs','c','NoFlows','orig','dest','size','deadline','tightness','MinAlpha'])
#number of instances 
NoInstances=10

In [4]:
#initialz network paramteres
units=[2]
nodes=['one', 'two', 'three', 'four', 'five', 'six']; 
arcs=tuplelist([('one','two'),('one' ,'three'),('two', 'one'),('two','four'),('two','five'),
      ('three', 'one'),( 'three', 'four'),('three', 'five' ),('four','two'),
      ('four','three'),('four','six'),('five','two'),('five','three'),('five','six'),('six','four'),('six','five')])
c={}
for arc in arcs:
    c[arc]=10 
NoFlows=5
orig=['','','','','','']
dest=['','','','','','']
size=[0.0,0.0,0.,0.0,0.0]
deadline=[3.300000,2.900000,4.700000,1.200000,0.033333]
deadline1=[3.300000,2.900000,4.700000,1.200000,0.033333]
tightness=[0,30]
#generating random instances and store them in data frame instances
for n in range(1,NoInstances+1):
    for f in range(NoFlows):
        rand1=random.randint(0,5)
        rand2=random.randint(0,5)
        while rand1==rand2:
            rand2=random.randint(0,5)
        orig[f]=nodes[rand1]
        dest[f]=nodes[rand2]
        size[f]=random.randint(1,20)
        deadline[f]=random.randint(1,20)
        MinAlpha=random.uniform(1.5, 4.5)
    instances.loc[len(instances)] = [units,nodes,arcs,c,NoFlows,orig[:],dest[:],
                                     size[:],deadline[:],tightness[:],MinAlpha]

In [5]:
#load each instance and solve it two times: one for a tight deadline and the other for a moderate one
#the result of each instance will be stored in data frame results.
for ins  in range(len(instances)):
    units=instances.iloc[ins]['units']
    nodes=instances.iloc[ins]['nodes']
    arcs=instances.iloc[ins]['arcs']
    c=instances.iloc[ins]['c'] 
    NoFlows=instances.iloc[ins]['NoFlows']
    orig=instances.iloc[ins]['orig']
    dest =instances.iloc[ins]['dest']
    size=instances.iloc[ins]['size']
    deadline=instances.iloc[ins]['deadline']
    tightness=instances.iloc[ins]['tightness']
    MinAlpha=instances.iloc[ins]['MinAlpha']
    size1=np.zeros(NoFlows)
    #creating model: 
    m = Model('net_flow')
    m.setParam('OutputFlag', 0)
    # Create variables
    r = m.addVars(range(NoFlows), name="r",lb=0,ub=float('inf'))
    y = m.addVars(range(NoFlows),arcs, name="y",lb=0)
    z = m.addVars(range(NoFlows),arcs,units, name="z",lb=0,vtype=GRB.INTEGER)
    #create constraints capacity type II
    capacity1=m.addConstrs(
    (sum(u*z[f,i,j,u] for u in units)>=y[f,i,j] for f in range(NoFlows) for i,j in arcs ), "capacity1")
    #create constraints capacity type I
    capacity2=m.addConstrs(
    (sum(u*z[f,i,j,u] for u in units for f in range(NoFlows))<=c[i,j] for i,j in arcs ), "capacity2")
    #create constraints origins
    m.addConstrs(
    (y.sum(f,'*',j)-y.sum(f,j,'*')==-r[f] for f in range(NoFlows) for j in [orig[f]] ), "balance_orig")
    #create constraints destinations
    m.addConstrs(
    (y.sum(f,'*',j)-y.sum(f,j,'*')==r[f] for f in range(NoFlows) for j in [dest[f]] ), "balance_dest")
    #create constraints blance
    m.addConstrs((y.sum(f,'*',j)-y.sum(f,j,'*')==0 
     for f in range(NoFlows) for j in list(set(nodes)-set().union([orig[f]],[dest[f]])) ), "balance_other")
    m.write('netflo1.lp')
    print('model is created.')
    print('***********************************\n')
    print('Instance No:%4d\n'%ins)
    print('***********************************\n')
    for t in tightness:
        print('\n')
        alpha=(MinAlpha+t*MinAlpha/100)
        deadline1= [d * alpha for d in deadline]
        makespan=0
        feasible=0
        size1=size[:]
        print('size=',size)
        print('original deadline=',deadline)
        print('deadline=',deadline1)
        print('makespan:%8.3f'%makespan)

        for iter in range(NoFlows):
            print('-------------------------------------------------------')
            print('instance:%4d , iter:%3d , tightness:%8.3f , alpha:%8.3f'%(ins,iter,t,alpha))
            print('-------------------------------------------------------')
            print('solve for size:', size1)
            m.update()
            #objective function
            m.setObjective(sum((1/deadline1[f]**2)*r[f] for f in range(NoFlows)), GRB.MAXIMIZE)
            for f in range(NoFlows):
                if size1[f]<0.0001:
                    r[f].ub=0
                else:
                    r[f].ub=float('inf')
            m.optimize()
            print('obj:%8.3f'%m.objVal)
            m.write('netflow.lp')
            print('model solved...')
            time=float('inf')*np.ones(NoFlows)
            rate=np.zeros(NoFlows)
            for f in range(NoFlows):
                if r[f].x>0:
                    rate[f]=r[f].x
                    time[f]=size1[f]/r[f].x
            print('rate=',rate)
            print('time=',time)
            print('min_time:%8.3f in index:%8.3f with deadline:%8.3f'%(min(time) ,np.argmin(time),deadline1[np.argmin(time)]))
            size1=size1-min(time)*rate
            makespan+=min(time)
            if makespan<=deadline1[np.argmin(time)]:
                print('feasible')
                feasible=1
            else:
                print('infeasible(makespan>deadline): %8.3f > %8.3f'%(makespan,deadline1[np.argmin(time)]))
                feasible=0
                break
            print('updating size:',size1)
            print('makespan:%8.3f'%makespan)
        if feasible==1:
            print('***problem feasible with makespan:%8.3f***'%makespan)
            results.loc[len(results)] = [ins,t,alpha,makespan]

        else:
            print('***problem INFEASIBLE***')
            results.loc[len(results)] = [ins,t,alpha,float('inf')]
results.to_excel('C:\Users\Ghafour Ahani\Dropbox\Ad\PythonProject\out.xlsx')

Academic license - for non-commercial use only
model is created.
***********************************

Instance No:   0

***********************************



('size=', [13, 1, 3, 13, 14])
('original deadline=', [5, 18, 18, 4, 1])
('deadline=', [22.349520902377442, 80.45827524855879, 80.45827524855879, 17.879616721901954, 4.469904180475488])
makespan:   0.000
-------------------------------------------------------
instance:   0 , iter:  0 , tightness:   0.000 , alpha:   4.470
-------------------------------------------------------
('solve for size:', [13, 1, 3, 13, 14])
obj:   1.542
model solved...
('rate=', array([20.,  0.,  0.,  0., 30.]))
('time=', array([0.65      ,        inf,        inf,        inf, 0.46666667]))
min_time:   0.467 in index:   4.000 with deadline:   4.470
feasible
('updating size:', array([ 3.66666667,  1.        ,  3.        , 13.        ,  0.        ]))
makespan:   0.467
-------------------------------------------------------
instance:   0 , iter:  1 , tightness

('updating size:', array([0., 0., 0., 0., 0.]))
makespan:   1.267
***problem feasible with makespan:   1.267***


('size=', [6, 16, 9, 18, 20])
('original deadline=', [5, 2, 11, 3, 20])
('deadline=', [20.080475825735398, 8.03219033029416, 44.17704681661788, 12.04828549544124, 80.32190330294159])
makespan:   0.000
-------------------------------------------------------
instance:   1 , iter:  0 , tightness:  30.000 , alpha:   4.016
-------------------------------------------------------
('solve for size:', [6, 16, 9, 18, 20])
obj:   0.479
model solved...
('rate=', array([10., 20., 10., 20., 10.]))
('time=', array([0.6, 0.8, 0.9, 0.9, 2. ]))
min_time:   0.600 in index:   0.000 with deadline:  20.080
feasible
('updating size:', array([ 0.,  4.,  3.,  6., 14.]))
makespan:   0.600
-------------------------------------------------------
instance:   1 , iter:  1 , tightness:  30.000 , alpha:   4.016
-------------------------------------------------------
('solve for size:', array([ 0.,  4.,  3

('rate=', array([20.,  0.,  0.,  0.,  0.]))
('time=', array([0.5625,    inf,    inf,    inf,    inf]))
min_time:   0.562 in index:   0.000 with deadline:  47.146
feasible
('updating size:', array([0., 0., 0., 0., 0.]))
makespan:   1.300
***problem feasible with makespan:   1.300***
model is created.
***********************************

Instance No:   3

***********************************



('size=', [13, 18, 12, 11, 1])
('original deadline=', [2, 9, 1, 19, 20])
('deadline=', [6.963189998604604, 31.33435499372072, 3.481594999302302, 66.15030498674373, 69.63189998604604])
makespan:   0.000
-------------------------------------------------------
instance:   3 , iter:  0 , tightness:   0.000 , alpha:   3.482
-------------------------------------------------------
('solve for size:', [13, 18, 12, 11, 1])
obj:   2.683
model solved...
('rate=', array([10.,  0., 30.,  0., 10.]))
('time=', array([1.3, inf, 0.4, inf, 0.1]))
min_time:   0.100 in index:   4.000 with deadline:  69.632
feasible
('

min_time:   0.200 in index:   0.000 with deadline:  11.815
feasible
('updating size:', array([ 0. ,  0. ,  0. , 12.5,  0. ]))
makespan:   0.950
-------------------------------------------------------
instance:   5 , iter:  4 , tightness:  30.000 , alpha:   2.363
-------------------------------------------------------
('solve for size:', array([ 0. ,  0. ,  0. , 12.5,  0. ]))
obj:   0.037
model solved...
('rate=', array([ 0.,  0.,  0., 30.,  0.]))
('time=', array([       inf,        inf,        inf, 0.41666667,        inf]))
min_time:   0.417 in index:   3.000 with deadline:  28.356
feasible
('updating size:', array([0., 0., 0., 0., 0.]))
makespan:   1.367
***problem feasible with makespan:   1.367***
model is created.
***********************************

Instance No:   6

***********************************



('size=', [8, 7, 9, 3, 13])
('original deadline=', [14, 19, 1, 11, 2])
('deadline=', [44.96108412519075, 61.01861416990174, 3.2115060089421967, 35.32656609836416, 6.4230120178843

makespan:   0.917
-------------------------------------------------------
instance:   7 , iter:  3 , tightness:   0.000 , alpha:   2.484
-------------------------------------------------------
('solve for size:', array([15.        ,  0.        ,  3.33333333,  0.        ,  0.        ]))
obj:   0.016
model solved...
('rate=', array([20.,  0., 10.,  0.,  0.]))
('time=', array([0.75      ,        inf, 0.33333333,        inf,        inf]))
min_time:   0.333 in index:   2.000 with deadline:  44.719
feasible
('updating size:', array([8.33333333, 0.        , 0.        , 0.        , 0.        ]))
makespan:   1.250
-------------------------------------------------------
instance:   7 , iter:  4 , tightness:   0.000 , alpha:   2.484
-------------------------------------------------------
('solve for size:', array([8.33333333, 0.        , 0.        , 0.        , 0.        ]))
obj:   0.011
model solved...
('rate=', array([20.,  0.,  0.,  0.,  0.]))
('time=', array([0.41666667,        inf,        in

('updating size:', array([0., 0., 0., 0., 0.]))
makespan:   1.100
***problem feasible with makespan:   1.100***
model is created.
***********************************

Instance No:   9

***********************************



('size=', [8, 6, 10, 1, 2])
('original deadline=', [2, 7, 3, 11, 18])
('deadline=', [5.812231308134296, 20.342809578470035, 8.718346962201444, 31.967272194738626, 52.31008177320866])
makespan:   0.000
-------------------------------------------------------
instance:   9 , iter:  0 , tightness:   0.000 , alpha:   2.906
-------------------------------------------------------
('solve for size:', [8, 6, 10, 1, 2])
obj:   1.044
model solved...
('rate=', array([30., 10., 10.,  0.,  0.]))
('time=', array([0.26666667, 0.6       , 1.        ,        inf,        inf]))
min_time:   0.267 in index:   0.000 with deadline:   5.812
feasible
('updating size:', array([0.        , 3.33333333, 7.33333333, 1.        , 2.        ]))
makespan:   0.267
------------------------------------

In [6]:
results

,Instance,Tightness,Alpha,Makespan
0,0.0,0.0,4.469904,1.450000
1,0.0,30.0,5.810875,1.450000
2,1.0,0.0,3.089304,1.266667
3,1.0,30.0,4.016095,1.266667
4,2.0,0.0,2.417718,1.300000
5,2.0,30.0,3.143034,1.300000
6,3.0,0.0,3.481595,1.600000
7,3.0,30.0,4.526073,1.600000
8,4.0,0.0,2.696831,1.325000
9,4.0,30.0,3.505880,1.325000
